# Comparación con el resto del mundo
> Como vienen evolucionando los paises donde COVID-19 llego antes en comparacion con Argentina?

- comments: false
- author: Pablo Zivic
- categories: [graficos]
- image: images/argentina-en-contexto.png
- permalink: /argentina-en-contexto/

In [1]:
#hide
import global_data

In [2]:
#hide
df = global_data.get_global_covid_df()

In [3]:
#hide
confirmed_by_country = df.groupby('country').confirmed.max().sort_values(ascending=False)
current_growth_rate_by_country = (
    df[df.confirmed > 1000]
      .groupby('country') 
      .confirmed_growth_rate.max()
      .sort_values(ascending=False)
)

In [4]:
#hide
import altair as alt

def plot_rate_cnt_trajectories(df, countries, selected):
    selection = alt.selection_multi(
        fields=['country'], bind='legend', init=[{'country': c} for c in selected]
    )
    
    source = df[(df.confirmed > 30) & (df.country.isin(countries))]
    
    return (
        alt.Chart(source)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X('confirmed', scale=alt.Scale(type='log'), axis=alt.Axis(title='Confirmed Cases')),
               y=alt.Y('confirmed_growth_rate', axis=alt.Axis(title='Rate of growth'), scale=alt.Scale(domain=[0.95, 1.65])),
               color=alt.Color('country:N', sort=selected),
               tooltip=list(df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200).properties(width=650, height=400)
    )

## Argentina comparado con el resto del mundo

**Shift + Click** sobre la leyenda para agregar o quitar paises

In [5]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 
]

with_most_cases = list(confirmed_by_country.index[:7])
controlled_situations = list(current_growth_rate_by_country[current_growth_rate_by_country < 1.1].index)[:7]

plot_rate_cnt_trajectories(
    df,
    with_most_cases + controlled_situations + latam, 
    selected=(
        ['Argentina'] + 
        list(current_growth_rate_by_country[current_growth_rate_by_country < 1.05].index) + 
        with_most_cases[:3]
    )
)

alt.Chart(...)

- Eje X: Cantidad de casos confirmados
- Eje Y: Tasa diaria de crecimiento

# Mirando al resto de latino america

Comparando con algunos paises de fuera de latam

In [6]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 'Japan', 'US'
]

plot_rate_cnt_trajectories(df, latam, ['Argentina', 'Chile', 'Brazil', 'Ecuador', 'Japan', 'US'])

alt.Chart(...)

Datos obtenidos de https://github.com/CSSEGISandData/COVID-19